# RFM Segmentation

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn

import warnings
warnings.filterwarnings('ignore')

import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
retail_df = pd.read_csv("https://raw.githubusercontent.com/manaranjanp/ISB_MLUL/main/clustering/retail_txns.csv")

In [ ]:
retail_df.Country.value_counts()

In [ ]:
retail_df.Country.value_counts()

In [ ]:
retail_df.shape

In [ ]:
retail_df.info();

In [ ]:
retail_df.isnull().sum()

In [ ]:
retail_df.dropna( subset = ['CustomerID'], inplace = True)

In [ ]:
retail_df.info()

In [ ]:
sn.kdeplot(retail_df.Quantity);

In [ ]:
retail_df[retail_df.Quantity < 0][0:10]

In [ ]:
sn.kdeplot(retail_df.UnitPrice);

## Calculating the order value

In [ ]:
retail_df['amount'] = retail_df.UnitPrice * retail_df.Quantity

In [ ]:
retail_df[0:5]

### RFM

In [ ]:
invoices_df = retail_df.groupby(['InvoiceNo', 'InvoiceDate', 'CustomerID'])['amount'].sum().reset_index()

In [ ]:
invoices_df.head(10)

In [ ]:
invoices_df.shape

In [ ]:
invoices_df['InvoiceDate'] = pd.to_datetime(invoices_df['InvoiceDate'])

In [ ]:
invoices_df.sample(10)

### Which month the invoice is created

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
def getDiffInMonths(now, since):
    
    difference = relativedelta(now, since)
    return difference.months + difference.years * 12

In [ ]:
max(invoices_df.InvoiceDate)

In [ ]:
last_month = max(invoices_df.InvoiceDate).date().replace(day=1)

In [ ]:
last_month

In [ ]:
invoices_df['monthsBefore'] =  invoices_df.apply(lambda rec:
                                    getDiffInMonths(last_month,
                                                    rec['InvoiceDate']),
                                                 axis = 1)

In [ ]:
invoices_df

In [ ]:
cust_grouping  = invoices_df.groupby(['CustomerID'])['monthsBefore']
invoices_df['recency'] = cust_grouping.transform('min')

In [ ]:
invoices_df.sample(10)

In [ ]:
invoices_df.shape

In [ ]:
recency_df = invoices_df[['CustomerID', 'recency']].drop_duplicates()

In [ ]:
recency_df.shape

## Calculating Frequency and Monetary Value


In [ ]:
frequency_df = invoices_df.CustomerID.value_counts().reset_index()

In [ ]:
frequency_df.columns = ['CustomerID', 'frequency']
frequency_df

In [ ]:
mvalue_df = invoices_df.groupby(['CustomerID'])['amount'].sum().reset_index()
mvalue_df.columns = ['CustomerID', 'mvalue']
mvalue_df

In [ ]:
rfm_df = recency_df.merge(frequency_df, on = 'CustomerID').merge(mvalue_df, on = 'CustomerID')

In [ ]:
rfm_df

In [ ]:
sn.scatterplot(data = rfm_df,
               x = 'frequency',
               y = 'mvalue');

In [ ]:
rfm_df['frequency'].describe()

In [ ]:
rfm_df['recency'].describe()

In [ ]:
rfm_df['f_bin'] = pd.cut( rfm_df['frequency'], [0, 3, 6, 50], labels = ['Low', 'Medium', 'High'])
rfm_df['r_bin'] = pd.cut( rfm_df['recency'], [-1, 1, 4, 20], labels =['High', 'Medium', 'Low'])

In [ ]:
rfm_df

In [ ]:
rfm_segments = pd.crosstab(rfm_df.f_bin,
                           rfm_df.r_bin,
                           normalize = "all")

In [ ]:
rfm_segments = rfm_segments.reindex(['High', 'Medium', 'Low'])

In [ ]:
sn.heatmap(rfm_segments[['Low', 'Medium', 'High']],
           annot = True,
           fmt = "0.3f",
           cmap = "crest");
plt.xlabel("Recency")
plt.ylabel("Frequency");

In [ ]:
sn.kdeplot(rfm_df.frequency);

In [ ]:
sn.boxplot(np.log(rfm_df.frequency));

In [ ]:
sn.kdeplot(rfm_df.mvalue);

In [ ]:
sn.boxplot(np.log(rfm_df.mvalue));

In [ ]:
rfm_df['log_frequency'] = np.log(rfm_df.frequency)

In [ ]:
rfm_df['log_mvalue'] = np.log(rfm_df.mvalue)

In [ ]:
rfm_df.frequency.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_rfm_df = scaler.fit_transform( rfm_df[['recency', 
                                              'log_frequency', 
                                              'log_mvalue']] )

In [ ]:
scaled_rfm_df[0:10]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
cluster_range = range( 2, 10 )
cluster_errors = []

for num_clusters in cluster_range:
  clusters = KMeans( num_clusters )
  clusters.fit( scaled_rfm_df )
  cluster_errors.append( clusters.inertia_ )

plt.figure(figsize=(8,4))
plt.plot( cluster_range, cluster_errors, marker = "o" );

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

fig, ax = plt.subplots(2, 2, figsize=(15,10))
num_clusters = [5, 6, 7, 8]
for i, k in enumerate(num_clusters):
    km = KMeans(n_clusters=k, 
                random_state=42)
    q, mod = divmod(i, 2)
    visualizer = SilhouetteVisualizer(km, 
                                      colors='yellowbrick', 
                                      ax=ax[q-1][mod])
    visualizer.fit(scaled_rfm_df) 

In [ ]:
k = 5

clusters = KMeans( k, random_state = 42 )
clusters.fit( scaled_rfm_df )
rfm_df["clusterid"] = clusters.labels_

In [ ]:
plt.figure(figsize = (10, 6))
sn.scatterplot(data = rfm_df,
               y = 'frequency',
               x = 'mvalue',
               hue = 'clusterid',
               style= "clusterid" );

In [ ]:
plt.figure(figsize = (10, 6))
sn.scatterplot(data = rfm_df,
               y = 'recency',
               x = 'mvalue',
               hue = 'clusterid',
               style= "clusterid" );

In [ ]:
sn.barplot(data = rfm_df,
           x = 'clusterid',
           y = 'mvalue',           
           estimator='mean');

In [ ]:
sn.barplot(data = rfm_df,
           x = 'clusterid',
           y = 'frequency');

In [ ]:
sn.barplot(data = rfm_df,
           x = 'clusterid',
           y = 'recency');

In [ ]:
rfm_df

In [ ]:
rfm_df[['recency', 'frequency', 'mvalue', 'clusterid']].groupby("clusterid").mean()

## Segmentation Interpretation:

- Cluster 0: New Customers

- Cluster 1: Loyal Customers and Promising

- Cluster 2: Star Customers

- Cluster 3: Dormant or Churned

- Cluster 4: On the Fence or Needs attention